## DS102 | Course Assignment
#### Name: Daryl Tan

### Research Topic 
The objective of this report is to understand if there is any correlation between Singapore's climate (air temperature, relative humidity and rainfall) and closing prices of Straits Times Index 

Being near the equator, Singapore is characterized by a hot and humid climate. The weather is warm and humid throughout the year and it rains on average 167 days a year. I am interested in this project as I am curious to know if a respite from this weather would have an effect on the closing prices in the stock market. As such, I decided to use the historical data of the Straits Times Index which is a market capitalisation weighted index that tracks the performances of the top 30 companies listed in the SGX.  


### Dataset 
1. Daily historical data of Straits Times Index in the form of a CSV file from stooq.com
2. Air temperature, humidity and rainfall readings from Jan 2017 to Dec 2018 across Singapore from data.gov.sg in the form of APIs

    
### Challenges 

1. Some data from data.gov was incomplete and did not contain values  
2. Having too many for loops led to my code taking around 60 hours to run 


In [1]:
# import libraries 
import pandas as pd 
import numpy as np 
import matplotlib
import datetime 
import requests 
import csv
import time
import json


In [3]:
# making API calls
number = 0
def get(url): 
    resp = requests.get(url)
    if resp.ok:
        return resp.json()
    

date = datetime.date(2016,12,31)
date_list = []
for i in range(730): # 730 is from jan 2017 to dec 2018 
    date += datetime.timedelta(days=1) # looping to increase days by 1
    start_url = f"https://api.data.gov.sg/v1/environment/rainfall?date={date}"
    date_list.append(date)
    data = get(start_url)
#     time.sleep(3)
    with open (f"{date}_rainfall.json", "w") as outfile: # saving into json file by date
        json.dump(data, outfile)
    number += 1 # checking for progress
    print (number)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [35]:
value_list = []
data_dict = {}
number = 0
filename = datetime.date(2016,12,31)
for i in range(730): # 730 is from jan 2017 to dec 2018 
    filename += datetime.timedelta(days=1) # looping to increase days by 1
    
    with open (f"{filename}.json") as f:
        data = json.load(f)
        for info in data["items"]:
            datetimestamp = info.get("timestamp")
            formatted_datetimestamp = (datetime.datetime.strptime(f"{datetimestamp}", "%Y-%m-%dT%H:%M:%S+08:00"))
            # filtering to data in working hours only 
            if (formatted_datetimestamp.hour) >= 9 and (formatted_datetimestamp.hour) < 17: 
                readings = [info.get("readings")]
                for dicts in readings[0]:
                    if (dicts.get("value") == None): #filtering out error in data where value is absent 
                        pass
                    else:                
                        value_list.append(dicts.get("value"))
                mean_temperature = round((sum(value_list)/len(value_list)), 2) #value per day. 50 stations in total
                data_dict[f"{filename}"] = [mean_temperature]
        number += 1 # checking for progress 
        print (number)
        

df = pd.DataFrame.from_dict(data_dict, orient="index", columns = ["Mean Temperature"])
df.to_csv("Mean Temperature By Day.csv")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [2]:
# reading mean humidity  
mean_humidity = pd.read_csv("Mean Humidity By Day.csv")
mean_humidity.head()

,Date,Mean Humidity
0,2017-01-01,77.51
1,2017-01-02,80.68
2,2017-01-03,80.82
3,2017-01-04,78.61
4,2017-01-05,75.79


In [3]:
# reading sti data 
sti = pd.read_csv("sti_data.csv")
sti.head()

,Date,Open,High,Low,Close,Volume
0,2017-01-03,2887.00,2899.67,2869.66,2898.97,174905552
1,2017-01-04,2897.18,2932.35,2897.18,2921.31,196293792
2,2017-01-05,2941.47,2968.15,2936.52,2954.14,262652256
3,2017-01-06,2964.82,2970.79,2954.86,2962.63,229369200
4,2017-01-09,2962.13,2986.49,2961.97,2981.54,157732992


In [4]:
# reading mean temp
mean_temp = pd.read_csv("Mean Temperature By Day.csv")
mean_temp.head()

,Date,Mean Temperature
0,2017-01-01,28.87
1,2017-01-02,28.25
2,2017-01-03,28.25
3,2017-01-04,28.32
4,2017-01-05,28.61


In [6]:
# reading mean rainfall
mean_rainfall = pd.read_csv("Mean Rainfall By Day.csv")
mean_rainfall.head()

,Date,Mean Rainfall
0,2017-01-01,6.59
1,2017-01-02,11.26
2,2017-01-03,2.39
3,2017-01-04,0.05
4,2017-01-05,0.01


In [7]:
temp_hum = mean_temp.merge(mean_humidity, on='Date')
temp_hum.to_csv("temp_hum.csv", index=False)

In [9]:
temp_hum_rain = temp_hum.merge(mean_rainfall, on="Date")
temp_hum_rain.to_csv("temp_hum_rain.csv", index=False)


In [10]:
temp_hum_rain_sti = temp_hum_rain.merge(sti, on="Date")
temp_hum_rain_sti.to_csv("temp_hum_rain_sti.csv", index=False)

In [11]:
# removing redundant columns 
data = temp_hum_rain_sti.drop(columns=["High", "Low", "Volume", "Open"])
data.head()

,Date,Mean Temperature,Mean Humidity,Mean Rainfall,Close
0,2017-01-03,28.25,80.82,2.39,2898.97
1,2017-01-04,28.32,78.61,0.05,2921.31
2,2017-01-05,28.61,75.79,0.01,2954.14
3,2017-01-06,28.95,73.99,0.14,2962.63
4,2017-01-09,29.74,70.11,0.00,2981.54


In [12]:
# new csv containing date, closing price and mean temp
data.to_csv("data.csv", index=False)